In [ ]:
!python -m pip install --upgrade pip
!python -m pip install accelerate>=0.20.1 -U
!python -m pip install transformers[torch] -U

import os

os.kill(os.getpid(), 9)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 28.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 98.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [1]:
import nltk
from torch.nn.modules import padding

nltk.download('stopwords')
from wordcloud import WordCloud, STOPWORDS
import tensorflow as tf
import matplotlib.pyplot as plt
import tarfile
import gzip
import os
from collections import defaultdict
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, GRU, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np
import random
from nltk.corpus import wordnet
from transformers import TrainingArguments

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yannp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Proj

## ======= Charger les données =======
Nous allons tout d'abord importer les données.

In [152]:
onDrive = False

if onDrive:
    from google.colab import drive

    drive.mount('/content/drive/')

In [153]:


!cp /content/drive/MyDrive/AI/Data/Common\ Language.zip /content/

'cp' n'est pas reconnu en tant que commande interne
ou externe, un programme ex‚cutable ou un fichier de commandes.


In [154]:
import zipfile


def unzip_file(zip_file_path, extract_to_path):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to_path)


if onDrive:
    unzip_file("/content/Common Language.zip", "/content/")
else:
    unzip_file("./Data/Common Language.zip", "./")

In [161]:
import os
import pandas as pd

# Chemin du dossier principal
if onDrive:
    base_path = "/content/common_voice_kpd"
else:
    base_path = "./common_voice_kpd"

# Initialiser des dataframes vides
train_df = pd.DataFrame()
test_df = pd.DataFrame()
dev_df = pd.DataFrame()

languages_to_include = ['English', 'French']

if onDrive:
    for language_folder in os.listdir(base_path):
        if language_folder in languages_to_include:
            folder_path = os.path.join(base_path, language_folder)
            if os.path.isdir(folder_path):
                # Chemin des fichiers
                train_file = os.path.join(folder_path, 'train.csv')
                test_file = os.path.join(folder_path, 'test.csv')
                dev_file = os.path.join(folder_path, 'dev.csv')

                # Lire et ajouter la colonne de langue, mettre à jour la colonne 'path'
                if os.path.exists(train_file):
                    temp_df = pd.read_csv(train_file, sep='\t', encoding='utf-16')
                    temp_df['language'] = language_folder
                    temp_df['path'] = folder_path + '/train/' + temp_df['client_id'] + "/" + temp_df[
                        'path']  # Mettre à jour la colonne 'path'
                    train_df = pd.concat([train_df, temp_df])

                if os.path.exists(test_file):
                    temp_df = pd.read_csv(test_file, sep='\t', encoding='utf-16')
                    temp_df['language'] = language_folder
                    temp_df['path'] = folder_path + '/test/' + temp_df['client_id'] + "/" + temp_df['path']
                    test_df = pd.concat([test_df, temp_df])

                if os.path.exists(dev_file):
                    temp_df = pd.read_csv(dev_file, sep='\t', encoding='utf-16')
                    temp_df['language'] = language_folder
                    temp_df['path'] = folder_path + '/dev/' + temp_df['client_id'] + "/" + temp_df['path']
                    dev_df = pd.concat([dev_df, temp_df])
else:
    for language_folder in os.listdir(base_path):
        if language_folder in languages_to_include:
            folder_path = os.path.join(base_path, language_folder)
            if os.path.isdir(folder_path):
                # Chemin des fichiers
                train_file = os.path.join(folder_path, 'train.csv')
                test_file = os.path.join(folder_path, 'test.csv')
                dev_file = os.path.join(folder_path, 'dev.csv')

                # Lire et ajouter la colonne de langue, mettre à jour la colonne 'path'
                if os.path.exists(train_file):
                    temp_df = pd.read_csv(train_file, sep='\t', encoding='utf-16')
                    temp_df['language'] = language_folder
                    temp_df['path'] = folder_path + '/train/' + temp_df['client_id'] + "/" + temp_df[
                        'path']  # Mettre à jour la colonne 'path'
                    train_df = pd.concat([train_df, temp_df])

                if os.path.exists(test_file):
                    temp_df = pd.read_csv(test_file, sep='\t', encoding='utf-16')
                    temp_df['language'] = language_folder
                    temp_df['path'] = folder_path + '/test/' + temp_df['client_id'] + "/" + temp_df['path']
                    test_df = pd.concat([test_df, temp_df])

                if os.path.exists(dev_file):
                    temp_df = pd.read_csv(dev_file, sep='\t', encoding='utf-16')
                    temp_df['language'] = language_folder
                    temp_df['path'] = folder_path + '/dev/' + temp_df['client_id'] + "/" + temp_df['path']
                    dev_df = pd.concat([dev_df, temp_df])

if not onDrive:
    # replace all \ with / in path column
    train_df['path'] = train_df['path'].str.replace('\\', '/')
    test_df['path'] = test_df['path'].str.replace('\\', '/')
    dev_df['path'] = dev_df['path'].str.replace('\\', '/')

# Suppression de colonnes inutiles
columns_to_remove = ["client_id", "age", "gender", "Unnamed: 0"]
train_df = train_df.drop(columns=columns_to_remove, errors='ignore')
test_df = test_df.drop(columns=columns_to_remove, errors='ignore')
dev_df = dev_df.drop(columns=columns_to_remove, errors='ignore')

# Réinitialiser les index des dataframes
train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)
dev_df.reset_index(drop=True, inplace=True)

concatenated_df = pd.concat([train_df, test_df, dev_df], ignore_index=True)

# Split the concatenated dataframe into train and test sets
train_df, test_df = train_test_split(concatenated_df, test_size=0.2, random_state=42)

# Reset index for both train and test sets
train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

In [162]:
train_df

,path,sentence,language
0,./common_voice_kpd/French/test/frn_tst_sp_84/c...,Elles ont exprimé leur soutien à cette expérim...,French
1,./common_voice_kpd/English/train/eng_trn_sp_66...,I left it downstairs.,English
2,./common_voice_kpd/French/train/frn_trn_sp_635...,S'en suivent cinq années de régime militaire.,French
3,./common_voice_kpd/French/train/frn_trn_sp_188...,Il monte aussi sur quatre autres podiums cette...,French
4,./common_voice_kpd/French/test/frn_tst_sp_77/c...,Chacun de ces deux cercles avaient créé un pri...,French
...,...,...,...
965,./common_voice_kpd/English/dev/eng_dev_sp_13/c...,There were to be no political parties in the n...,English
966,./common_voice_kpd/English/dev/eng_dev_sp_63/c...,The farm is open to tourists.,English
967,./common_voice_kpd/French/dev/frn_dev_sp_28/co...,Le titre a également été vacant à une reprise.,French
968,./common_voice_kpd/English/test/eng_tst_sp_16/...,It is part of Saint Andrew's Parish.,English


In [163]:
test_df

,path,sentence,language
0,./common_voice_kpd/English/train/eng_trn_sp_67...,All are native to Nebraska.,English
1,./common_voice_kpd/French/train/frn_trn_sp_250...,Foyle est diplômé de l'université Colgate en h...,French
2,./common_voice_kpd/English/train/eng_trn_sp_51...,The sun shines on both sides of the hedge.,English
3,./common_voice_kpd/English/train/eng_trn_sp_46...,I could go on for days about the delicious win...,English
4,./common_voice_kpd/French/train/frn_trn_sp_672...,C'est l'origine de la Schackgalerie de Munich.,French
...,...,...,...
238,./common_voice_kpd/English/train/eng_trn_sp_79...,The borough was named for the area's terrain.,English
239,./common_voice_kpd/English/test/eng_tst_sp_68/...,It is an ethical abomination because it is vio...,English
240,./common_voice_kpd/English/train/eng_trn_sp_67...,He is officially a candidate for the United St...,English
241,./common_voice_kpd/French/train/frn_trn_sp_43/...,Ce n’est pas uniquement pour accumuler des fon...,French


## ======= première analyse des données =======

nous cherchons d'abord la taille de l'ensemble de données

In [196]:
import re
import unicodedata

chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"”_]'


def remove_special_characters(text):
    # Remove all characters that are not letters or numbers
    text = re.sub(r'[^a-zA-Z0-9 éèà]', '', text)

    # Convert to lowercase
    text = text.lower()

    return text

In [197]:
train_df['sentence'] = train_df['sentence'].apply(remove_special_characters)
test_df['sentence'] = test_df['sentence'].apply(remove_special_characters)


In [198]:
def extract_all_chars(df):
    all_text = ' '.join(df['sentence'])
    vocab = list(set(all_text))
    return {"vocab": vocab, "all_text": all_text}


# Appliquer la fonction sur l'ensemble du DataFrame
vocabs_train = extract_all_chars(train_df)
vocabs_test = extract_all_chars(test_df)

In [199]:
vocab_list = list(set(vocabs_train['vocab']) | set(vocabs_test['vocab']))
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'m': 0,
 'é': 1,
 'e': 2,
 'c': 3,
 'l': 4,
 'z': 5,
 'g': 6,
 'à': 7,
 'i': 8,
 ' ': 9,
 'h': 10,
 'b': 11,
 'w': 12,
 'p': 13,
 'u': 14,
 'f': 15,
 'y': 16,
 'k': 17,
 's': 18,
 't': 19,
 'n': 20,
 'r': 21,
 'j': 22,
 'a': 23,
 'v': 24,
 'è': 25,
 'q': 26,
 'o': 27,
 'x': 28,
 'd': 29}

In [200]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [201]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
print(len(vocab_dict))
vocab_dict

32


{'m': 0,
 'é': 1,
 'e': 2,
 'c': 3,
 'l': 4,
 'z': 5,
 'g': 6,
 'à': 7,
 'i': 8,
 'h': 10,
 'b': 11,
 'w': 12,
 'p': 13,
 'u': 14,
 'f': 15,
 'y': 16,
 'k': 17,
 's': 18,
 't': 19,
 'n': 20,
 'r': 21,
 'j': 22,
 'a': 23,
 'v': 24,
 'è': 25,
 'q': 26,
 'o': 27,
 'x': 28,
 'd': 29,
 '|': 9,
 '[UNK]': 30,
 '[PAD]': 31}

In [202]:
import json

with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [203]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [204]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True,
                                             return_attention_mask=False)

In [205]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [206]:
import soundfile as sf
from tqdm import tqdm


def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = sf.read(batch["path"])
    batch["speech"] = speech_array
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["sentence"]
    return batch


# Appliquer la fonction sur chaque ligne du DataFrame
tqdm.pandas(desc="Processing1")
train_df = train_df.progress_apply(speech_file_to_array_fn, axis=1)
tqdm.pandas(desc="Processing2")
test_df = test_df.progress_apply(speech_file_to_array_fn, axis=1)

# Supprimer les colonnes spécifiques
columns_to_remove = ['path', 'sentence']  # Mettez ici les noms des colonnes à enlever
train_df = train_df.drop(columns=columns_to_remove)
test_df = test_df.drop(columns=columns_to_remove)



Processing2: 100%|██████████| 204/204 [00:00<00:00, 495.56it/s]


In [207]:
import IPython.display as ipd
import numpy as np
import random

# Sélectionner un index aléatoire
rand_int = random.randint(0, len(train_df) - 1)

# Lire les données audio de l'échantillon sélectionné
audio_data = train_df.iloc[rand_int]["speech"]
sampling_rate = train_df.iloc[rand_int]["sampling_rate"]  # Assurez-vous d'avoir cette colonne dans votre DataFrame

# Jouer l'audio
ipd.Audio(data=np.asarray(audio_data), autoplay=True, rate=sampling_rate)

In [208]:

# Selectionner un index aléatoire
rand_int = random.randint(0, len(test_df) - 1)

# Lire les données audio de l'échantillon sélectionné
audio_data = test_df.iloc[rand_int]["speech"]
sampling_rate = test_df.iloc[rand_int]["sampling_rate"]  # Assurez-vous d'avoir cette colonne dans votre DataFrame

# Jouer l'audio
ipd.Audio(data=np.asarray(audio_data), autoplay=True, rate=sampling_rate)

In [209]:
def prepare_dataset(row):
    # Assurer que l'échantillon a le bon taux d'échantillonnage
    assert row[
               "sampling_rate"] == processor.feature_extractor.sampling_rate, f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    # Traiter les données audio
    input_values = processor(row["speech"], sampling_rate=row["sampling_rate"]).input_values

    # Traiter les étiquettes
    labels = processor(text=row["target_text"]).input_ids

    row["input_values"] = input_values
    row["labels"] = labels
    return row


tqdm.pandas(desc="Processing")
# Appliquer la fonction sur chaque ligne du DataFrame
train_df_prepared = train_df.progress_apply(prepare_dataset, axis=1)
test_df_prepared = test_df.progress_apply(prepare_dataset, axis=1)

# Supprimer les colonnes inutiles
columns_to_remove = []
train_df_prepared = train_df_prepared.drop(columns=columns_to_remove)
test_df_prepared = test_df_prepared.drop(columns=columns_to_remove)


Processing: 100%|██████████| 204/204 [00:00<00:00, 459.71it/s]


In [210]:
train_df_prepared

,language,speech,sampling_rate,target_text,input_values,labels
0,English,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",16000,it is a charity school whose fees are calculat...,"[[0.0003061341, 0.0003061341, 0.0003061341, 0....","[8, 19, 9, 8, 18, 9, 23, 9, 3, 10, 23, 21, 8, ..."
1,English,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",16000,john looked out the window of the showhouse ha...,"[[0.00013979246, 0.00013979246, 0.00013979246,...","[22, 27, 10, 20, 9, 4, 27, 27, 17, 2, 29, 9, 2..."
2,English,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",16000,at least fortyone protesters were killed in th...,"[[3.9257215e-05, 3.9257215e-05, 3.9257215e-05,...","[23, 19, 9, 4, 2, 23, 18, 19, 9, 15, 27, 21, 1..."
3,English,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",16000,it was so small and undeveloped,"[[0.0017256176, 0.0017256176, 0.0017256176, 0....","[8, 19, 9, 12, 23, 18, 9, 18, 27, 9, 18, 0, 23..."
4,English,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",16000,do you mean it,"[[-0.00066482445, -0.00066482445, -0.000664824...","[29, 27, 9, 16, 27, 14, 9, 0, 2, 23, 20, 9, 8,..."
...,...,...,...,...,...,...
814,French,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",16000,trentequatre éléphants,"[[0.00013638378, 0.00013638378, 0.00013638378,...","[19, 21, 2, 20, 19, 2, 26, 14, 23, 19, 21, 2, ..."
815,French,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",16000,lordre du jour appelle les questions au gouver...,"[[-1.0867594e-05, -1.0867594e-05, -1.0867594e-...","[4, 27, 21, 29, 21, 2, 9, 29, 14, 9, 22, 27, 1..."
816,French,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",16000,la distance métrique commune pour des variable...,"[[0.00013176339, 0.00013176339, 0.00013176339,...","[4, 23, 9, 29, 8, 18, 19, 23, 20, 3, 2, 9, 0, ..."
817,French,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",16000,la sélection représente les paysbas aux champi...,"[[-0.0015854842, -0.0015854842, -0.0015854842,...","[4, 23, 9, 18, 1, 4, 2, 3, 19, 8, 27, 20, 9, 2..."


In [211]:
test_df_prepared

,language,speech,sampling_rate,target_text,input_values,labels
0,English,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",16000,he received the following cable from his offic...,"[[-9.09185e-05, -9.09185e-05, -9.09185e-05, -9...","[10, 2, 9, 21, 2, 3, 2, 8, 24, 2, 29, 9, 19, 1..."
1,English,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",16000,tack this picture to the wall,"[[-0.00024580627, -0.00024580627, -0.000245806...","[19, 23, 3, 17, 9, 19, 10, 8, 18, 9, 13, 8, 3,..."
2,English,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",16000,what would you do,"[[0.000801247, 0.000801247, 0.000801247, 0.000...","[12, 10, 23, 19, 9, 12, 27, 14, 4, 29, 9, 16, ..."
3,English,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",16000,he continued to use the seneca view camera for...,"[[-0.0009788852, -0.0009788852, -0.0009788852,...","[10, 2, 9, 3, 27, 20, 19, 8, 20, 14, 2, 29, 9,..."
4,English,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",16000,from trivial things great contests often arise,"[[0.0004450683, 0.0004450683, 0.0004450683, 0....","[15, 21, 27, 0, 9, 19, 21, 8, 24, 8, 23, 4, 9,..."
...,...,...,...,...,...,...
199,French,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",16000,qui les commandait boishardy,"[[9.728779e-05, 9.728779e-05, 9.728779e-05, 9....","[26, 14, 8, 9, 4, 2, 18, 9, 3, 27, 0, 0, 23, 2..."
200,French,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",16000,son thème favori est celui du mensonge,"[[0.0015968519, 0.0015968519, 0.0015968519, 0....","[18, 27, 20, 9, 19, 10, 25, 0, 2, 9, 15, 23, 2..."
201,French,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",16000,jusqualors toutes les capitulations nétaient a...,"[[0.00020545206, 0.00020545206, 0.00020545206,...","[22, 14, 18, 26, 14, 23, 4, 27, 21, 18, 9, 19,..."
202,French,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",16000,il est vrai que leur ressemblance avec les pei...,"[[4.9992123e-06, 4.9992123e-06, 4.9992123e-06,...","[8, 4, 9, 2, 18, 19, 9, 24, 21, 23, 8, 9, 26, ..."


In [212]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union


@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = []
        for feature in features:
            input_values = feature["input_values"]
            if len(input_values) == 1 and isinstance(input_values[0], np.ndarray):
                # If input_values is a list of a single numpy.ndarray, flatten it
                input_features.append({"input_values": input_values[0]})
            else:
                input_features.append({"input_values": input_values})

        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [213]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [214]:
!pip install datasets
!pip install jiwer
from datasets import load_metric

wer_metric = load_metric("wer")

/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for wer contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/wer/wer.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [215]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [216]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-xlsr-53-espeak-cv-ft",
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
)
model.gradient_checkpointing_enable()
print("Model's pad_token_id:", model.config.pad_token_id)

Some weights of the model checkpoint at facebook/wav2vec2-xlsr-53-espeak-cv-ft were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-xlsr-53-espeak-cv-ft and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should prob

Model's pad_token_id: 31


In [217]:
model.freeze_feature_extractor()

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1918: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5. Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [218]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./saves",
    # output_dir="./wav2vec2-base-timit-demo",
    group_by_length=True,
    per_device_train_batch_size=32,
    evaluation_strategy="steps",
    num_train_epochs=30,
    fp16=True,
    save_steps=100,
    eval_steps=100,
    logging_steps=100,
    learning_rate=1e-4,
    weight_decay=0.005,
    warmup_steps=200,
    save_total_limit=2,
)

In [219]:
from transformers import Trainer
from transformers import TrainerCallback
from tqdm.auto import tqdm

train_dataset = train_df_prepared.to_dict(orient='records')
eval_dataset = test_df_prepared.to_dict(orient='records')

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=processor.feature_extractor,
)


In [220]:
"""
function ConnectButton(){
    console.log("Connect pushed");
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click()
}
setInterval(ConnectButton,60000);
"""

'\nfunction ConnectButton(){\n    console.log("Connect pushed");\n    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click()\n}\nsetInterval(ConnectButton,60000);\n'

In [221]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
100,37.165200,6.869055,0.996986
200,7.613800,3.067161,1.000000
300,3.204700,2.975052,1.000000
400,3.029800,2.942833,1.000000
500,2.977300,2.860258,1.000000
600,2.863300,2.625516,1.006529
700,2.617500,2.234094,0.973882


/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Tran

TrainOutput(global_step=780, training_loss=7.874372159517728, metrics={'train_runtime': 717.4527, 'train_samples_per_second': 34.246, 'train_steps_per_second': 1.087, 'total_flos': 8.232098604231997e+18, 'train_loss': 7.874372159517728, 'epoch': 30.0})

In [222]:
processor = Wav2Vec2Processor.from_pretrained("patrickvonplaten/wav2vec2-base-timit-demo")

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:365: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [223]:
model = Wav2Vec2ForCTC.from_pretrained("patrickvonplaten/wav2vec2-base-timit-demo")

Some weights of the model checkpoint at patrickvonplaten/wav2vec2-base-timit-demo were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at patrickvonplaten/wav2vec2-base-timit-demo and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You shoul

In [224]:
def map_to_result(batch):
    model.to("cuda")
    input_values = processor(
        batch["speech"],
        sampling_rate=batch["sampling_rate"],
        return_tensors="pt"
    ).input_values.to("cuda")

    with torch.no_grad():
        logits = model(input_values).logits

    pred_ids = torch.argmax(logits, dim=-1)
    batch["pred_str"] = processor.batch_decode(pred_ids)[0]

    return batch

In [225]:
results = test_df_prepared.apply(map_to_result, axis=1)

In [226]:
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["target_text"])))

Test WER: 0.889


In [227]:
from IPython.display import display, HTML


def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = random.sample(range(len(dataset)), num_examples)

    df = dataset.iloc[picks].drop(columns=["speech", "sampling_rate", "input_values", "labels"])
    display(HTML(df.to_html()))


show_random_elements(results)

,language,target_text,pred_str
163,French,chacun de ces deux cercles avaient créé un prix littéraire récompensant un ouvrage historique,sho countiss i duicsec a victheopocuieteria hycomposale on tou largies tuek
28,English,despite the tantalizing possibilities laws hedges his bets and never cuts loose,dispite the tentalizing possibilities loawrs hedges his bets and never cuts looce
6,English,glue the sheet to the dark blue background,plue the sheat to the dark glue background
189,French,les armoiries du noble bénédictin figurent sur le linteau en pierre de la porte,iis hi ma you no revene o clme she yunu saw your locial yit beutat k
70,English,im going to tell your mother on you,anguies to tell you where are you
62,English,this approach often uses other media and breaks accepted conventions and practices,these upbroge often uses audormadia and rigs accept tha gonventions and bracthesis
57,English,as they packed their bags the tsunami sirens sounded once more across the bleak shores,as they backed their bags the sunami siderens soundered once more across the bleek sorts
35,English,ive been unable to do my job ever since i read the piece,handbing hng averof swe may ch of every since iru th plice
188,French,avi nash nait aux tatsunis,i venash ni was it as youn
26,English,i assume this was your doing,i assumed this was your doing


In [228]:
model.to("cuda")
input_values = processor(test_df_prepared.iloc[0]["speech"], sampling_rate=test_df_prepared.iloc[0]["sampling_rate"],
                         return_tensors="pt").input_values.to("cuda")

with torch.no_grad():
    logits = model(input_values).logits

pred_ids = torch.argmax(logits, dim=-1)

# convert ids to tokens
" ".join(processor.tokenizer.convert_ids_to_tokens(pred_ids[0].tolist()))

'[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] h h h e e | | | r r e e [PAD] [PAD] c c c [PAD] e e e i i v v v e e d d | t h h e e | | f f [PAD] o o l l [PAD] [PAD] l l o o w w w i i n n g g g | | | g [PAD] [PAD] [PAD] [PAD] a a a [PAD] [PAD] [PAD] b b [PAD] l l l l e [PAD] | | | [PAD] f r r r o o m m | | h h i s s s | | | [PAD] o o o f [PAD] [PAD] [PAD] f f f [PAD] i i [PAD] c c e e e [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] | | [PAD] a a r r [PAD] [PAD] l l [PAD] [PAD] i [PAD] n n n e e e [PAD] [PAD] b [PAD] a a [PAD] n n n [PAD] [PAD] [PAD] [PA

In [229]:
results_test = []
model.to("cuda")
for index, row in test_df_prepared.iterrows():
    # Process the input
    input_values = processor(row["speech"], sampling_rate=row["sampling_rate"], return_tensors="pt").input_values.to(
        "cuda")

    with torch.no_grad():
        # Pass both the input_values (on CUDA) and their lengths (on CPU) to the model
        logits = model(input_values).logits

    pred_ids = torch.argmax(logits, dim=-1)

    results_test.append((processor.tokenizer.convert_ids_to_tokens(pred_ids[0].tolist()), row["language"]))

results_train = []
model.to("cuda")
for index, row in test_df_prepared.iterrows():
    # Process the input
    input_values = processor(row["speech"], sampling_rate=row["sampling_rate"], return_tensors="pt").input_values.to(
        "cuda")

    with torch.no_grad():
        # Pass both the input_values (on CUDA) and their lengths (on CPU) to the model
        logits = model(input_values).logits

    pred_ids = torch.argmax(logits, dim=-1)

    results_train.append((processor.tokenizer.convert_ids_to_tokens(pred_ids[0].tolist()), row["language"]))


In [230]:
import pandas as pd

# Convert the 'results' list to a DataFrame
df_results_test = pd.DataFrame(results_test, columns=['phoneme', 'language'])

# Display the first few rows of the DataFrame to verify
print(df_results_test.head())


                                             phoneme language
0  [[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...  English
1  [[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...  English
2  [[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...  English
3  [[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...  English
4  [[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...  English


In [231]:
import pandas as pd

# Convert the 'results' list to a DataFrame
df_results_train = pd.DataFrame(results_train, columns=['phoneme', 'language'])

# Display the first few rows of the DataFrame to verify
print(df_results_train.head())


                                             phoneme language
0  [[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...  English
1  [[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...  English
2  [[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...  English
3  [[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...  English
4  [[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...  English


In [232]:
!cp /content/drive/MyDrive/AI/Data/merged_data.csv /content/

In [2]:
import pandas as pd
from os.path import exists

# Path to the CSV file
csv_file_path = 'merged_data.csv'

# Check if the CSV file exists
if exists(csv_file_path):
    # Load the DataFrame from the CSV file
    df_merged = pd.read_csv(csv_file_path)
    print("Loaded merged DataFrame from CSV.")
else:
    # Merge the DataFrames and shuffle
    df_merged = pd.concat([df_results_test, df_results_train], ignore_index=True)
    df_merged = df_merged.sample(frac=1).reset_index(drop=True)

    # Optionally, save the merged DataFrame to a CSV file for future use
    df_merged.to_csv(csv_file_path, index=False)
    print("Saved merged DataFrame to CSV.")

# Determine the split index
split_index = int(0.8 * len(df_merged))

# Split into training and testing datasets
train_df = df_merged[:split_index]
test_df = df_merged[split_index:]

print(f"Training set size: {len(train_df)}, Testing set size: {len(test_df)}")



Loaded merged DataFrame from CSV.
Training set size: 970, Testing set size: 243


In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import ast


def convertir_en_liste(chaine):
    try:
        # Convertit la chaîne en liste en utilisant ast.literal_eval
        return ast.literal_eval(chaine)
    except Exception as e:
        # Gérer les exceptions si la conversion échoue
        print(f"Erreur de conversion : {e}")
        return []


def reduire_pad(liste):
    nouvelle_liste = []
    for elem in liste:
        if elem != "[PAD]" or (nouvelle_liste and nouvelle_liste[-1] != "[PAD]"):
            nouvelle_liste.append(elem)
    return nouvelle_liste


# Appliquer la conversion et la réduction
# Assurez-vous que train_df est une copie indépendante si c'est un sous-ensemble
train_df = train_df.copy()

# Appliquer la conversion et la réduction en utilisant .loc
train_df.loc[:, 'phoneme'] = train_df['phoneme'].apply(lambda x: reduire_pad(convertir_en_liste(x)))

# Afficher les premières lignes de train_df pour vérifier les changements
print(train_df.head())

                                             phoneme language
0  [t, t, h, h, e, y, y, |, |, s, s, s, [PAD], o,...  English
1  [t, t, o, o, |, |, |, c, c, [PAD], e, e, l, l,...  English
2  [t, t, h, h, e, e, |, |, |, |, b, [PAD], o, o,...  English
3  [d, [PAD], o, o, [PAD], n, [PAD], o, u, w, [PA...   French
4  [h, e, e, |, |, |, c, [PAD], m, m, [PAD], o, [...  English


In [4]:
test_df = test_df.copy()

# Appliquer la conversion et la réduction en utilisant .loc
test_df.loc[:, 'phoneme'] = test_df['phoneme'].apply(lambda x: reduire_pad(convertir_en_liste(x)))

# Afficher les premières lignes de train_df pour vérifier les changements
print(test_df.head())

                                               phoneme language
970  [t, t, h, h, e, e, |, |, |, |, p, p, [PAD], a,...  English
971  [t, h, e, e, e, x, [PAD], p, p, e, [PAD], d, [...   French
972  [a, a, [PAD], |, |, y, y, o, o, u, u, |, |, |,...  English
973  [t, t, [PAD], o, o, [PAD], v, [PAD], e, e, s, ...  English
974  [w, h, h, a, t, |, |, d, o, o, e, [PAD], s, s,...  English


In [5]:
train_df['language'] = train_df['language'].apply(lambda x: 0 if x == 'French' else 1)
test_df['language'] = test_df['language'].apply(lambda x: 0 if x == 'French' else 1)

# Ensuite, continuez avec la tokenisation et le padding comme avant
phoneme_tokenizer = Tokenizer(char_level=False)
phoneme_tokenizer.fit_on_texts(train_df['phoneme'])
X_train = phoneme_tokenizer.texts_to_sequences(train_df['phoneme'])
X_train = pad_sequences(X_train, padding='post')

X_test = phoneme_tokenizer.texts_to_sequences(test_df['phoneme'])
X_test = pad_sequences(X_test, padding='post', maxlen=X_train.shape[1])

# Encode the language labels
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train_df['language'])
y_test = label_encoder.transform(test_df['language'])



In [6]:
print(test_df.head())

                                               phoneme  language
970  [t, t, h, h, e, e, |, |, |, |, p, p, [PAD], a,...         1
971  [t, h, e, e, e, x, [PAD], p, p, e, [PAD], d, [...         0
972  [a, a, [PAD], |, |, y, y, o, o, u, u, |, |, |,...         1
973  [t, t, [PAD], o, o, [PAD], v, [PAD], e, e, s, ...         1
974  [w, h, h, a, t, |, |, d, o, o, e, [PAD], s, s,...         1


In [7]:
print(train_df.head())

                                             phoneme  language
0  [t, t, h, h, e, y, y, |, |, s, s, s, [PAD], o,...         1
1  [t, t, o, o, |, |, |, c, c, [PAD], e, e, l, l,...         1
2  [t, t, h, h, e, e, |, |, |, |, b, [PAD], o, o,...         1
3  [d, [PAD], o, o, [PAD], n, [PAD], o, u, w, [PA...         0
4  [h, e, e, |, |, |, c, [PAD], m, m, [PAD], o, [...         1


In [33]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Parameters for the model
vocab_size = len(phoneme_tokenizer.word_index) + 1  # Vocabulary size
embed_dim = 1024  # Dimension of the embedding vectors
max_length = X_train.shape[1]  # Maximum length of the input sequences
output_dim = len(label_encoder.classes_)  # Number of unique output classes

# Define the RNN model
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embed_dim, input_length=max_length),
    LSTM(units=128, dropout=0.2, recurrent_dropout=0.2, return_sequences=True),
    LSTM(units=128, dropout=0.2, recurrent_dropout=0.2),
    Dense(output_dim, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()



Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, 640, 1024)         30720     
                                                                 
 lstm_10 (LSTM)              (None, 640, 128)          590336    
                                                                 
 lstm_11 (LSTM)              (None, 128)               131584    
                                                                 
 dense_8 (Dense)             (None, 2)                 258       
                                                                 
Total params: 752898 (2.87 MB)
Trainable params: 752898 (2.87 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, verbose=1, mode='max', restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train, epochs=150, validation_data=(X_test, y_test), batch_size=32,
                    callbacks=[early_stopping])

# Préparation de la gamme des époques pour le tracé.
epochs_range = range(1, len(history.history['accuracy']) + 1)

# Configuration de la taille et du layout des graphiques.
plt.figure(figsize=(12, 4))

# Tracé de la perte d'entraînement et de validation.
plt.subplot(1, 2, 1)
plt.plot(epochs_range, history.history['loss'], label='Training Loss')
plt.plot(epochs_range, history.history['val_loss'], label='Validation Loss')
plt.title('Loss')
plt.legend()

# Tracé de la précision d'entraînement et de validation.
plt.subplot(1, 2, 2)
plt.plot(epochs_range, history.history['accuracy'], label='Training Accuracy')
plt.plot(epochs_range, history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy')
plt.legend()

# Affichage des graphiques.
plt.tight_layout()
plt.show()


Epoch 1/150
 1/31 [..............................] - ETA: 5:43 - loss: 0.6938 - accuracy: 0.4375

In [ ]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy}')
